# Glove Embeddings Demonstration

This notebook is meant to demonstrate the features of Glove Embeddings to see how they can potentially be used in conjunction with the COCO dataset to numerically analyze explanations and such. The link to download the embeddings is [here](https://nlp.stanford.edu/projects/glove), and I downloaded the **6b** one with Wikipedia and all.

## Imports that may be Necessary

```python
from collections import defaultdict
import numpy as np
import gensim
from gensim.models.keyedvectors import KeyedVectors
from sklearn.decomposition import TruncatedSVD
import matplotlib.pyplot as plt
%matplotlib inline
```
*Note: You may need to conda install gensim*

In [1]:
#Imports Cell
from collections import defaultdict
import numpy as np
import gensim
from gensim.models.keyedvectors import KeyedVectors
from sklearn.decomposition import TruncatedSVD
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
#Put the path to glove here
path = r"C:/Users/Vishn/Downloads/glove.6B.50d.txt.w2v"

#Now load the model into the variable "glove" (may take some time)
glove = KeyedVectors.load_word2vec_format(path, binary=False)

## How to Use Glove
```python
glove["word"] # Will give glove embedding vector for the word

"word" in glove #Checks if word is in glove (acts like a dictionary

glove["husband"] - glove["man"] + glove["woman"] #Should give representation that is wife

#To find most similar term to a vector:
    
glove.similar_by_vector(query)

#More advanced way to do this

glove.most_similar_cosmul(positive=['husband', 'woman'], negative=['man'])

#Since they are vectors, we can find the distance using dot products
```


In [3]:
#Define our domains based on the labels for searching the images
#Suppose "car" and "train" domain
# Iteratively find the most similar (ex. depth of 2 = for each main label, also add all their friend labels)
glove.similar_by_vector(glove["stoplight"])

[('stoplight', 1.0000001192092896),
 ('flyspeck', 0.6498768329620361),
 ('one-horse', 0.6459125876426697),
 ('18-wheeler', 0.6452605128288269),
 ('sea-side', 0.615469217300415),
 ('nanopores', 0.613706111907959),
 ('tepee', 0.6063854694366455),
 ('12-block', 0.6035683751106262),
 ('abobo', 0.5967516303062439),
 ('hurriyah', 0.5955226421356201)]

In [4]:
#TODO: Simple parser entity extraction for the captions and their labels

# General Ideas for Symbolic Reasoning

Suppose we had a bunch of **labels** from each of the outputs for the subsystems (the sem. seg & the two captions), we try to figure out how close all the different labels are across different systems.

We figure out a threshold and if 2-3 of them are super close in their vectors, and another one is not, we suspect that one, generally, but here are some other general ideas:

- Since each subsystem will present its own set of labels, (all the labels must be relatively close to each other) if **any of them seem abnormaly far away from the others** (maybe we can scramble to see this), then we say that it is not reasonable
- We do the same thing across multiple ones as well, and see the distances (maybe min distances) and try to figure out at a high level who is not reasonable
- We combine these local, and high-level checks with symbolic checks to determine overall reasonability

# Testing Demonstration

Let's take 3 different images from the CoCo Dataset where:

- Two of the images will be similar, and 1 image will be different

We will see if we can use the **semantic segmentation tags** + some basic distance calculations to see which images should be closest to each other:

Links to the three images used:

- 
- 
- 

# Math for Calculating Distances Between Captions

Suppose we have **sem. seg** labels that identify **O** objects, assuming we are using **N** dimensional vectors, we have a (**O**x**N**) array representing our values. 

Now suppose we have **caption labels** that identify **K** objects, assuming we are still using **N** dimensional vectors, we have a (**K**x**N**) array representing our values. 

## Finding Distances

When we are trying to find the distances, we need to use **multiplication** to make this easy. However, we need to remember that order matters if we were to just blindly do it:

```python
[[man], [woman], [cat]] * [[man], [woman], [cat]] = 0 #As distances between each element and itself would be zero

#However
[[cat], [woman], [man]] * [[man], [woman], [cat]] !=0 #As order is different so what will get multiplied is different
```

Therefore we will specifically use **matrix multiplication**. Therefore, the steps to find distances based on this are:


The python code to do so is as follows (uses `numpy`):

```python
O = 7
N = 2
K = 6

#Create two matrixes based on dimensions
x = np.arange(14).reshape((O,N))
y = np.arange(12).reshape((K,N))

# Distances are x^2 + y^2 - 2*x*y
dists = np.sum(x**2, axis=1)[:,np.newaxis] + np.sum(y**2, axis=1) 
dists -= 2*np.matmul(x,y.T)

distances = np.sqrt(dists)

#Now we find the minimum of this to represent as our distance
#For the axis, select whichever axis is smaller (will either be zero or 1)


np.min(distances,axis = np.argmin(distances.shape))

#By doing the above, we cover the one with more objects, so that there is a greater chance of greater distances (vs not properly accounting for an object's distance to others. Though, it shouldn't really matter either way.

```

In [5]:
arr_a = np.array([glove["man"],glove["woman"],glove["cat"]])
arr_b = np.array([glove["man"],glove["woman"],glove["cat"],glove["dog"]])

a = np.sum(arr_a**2,axis = 1)[:,np.newaxis]
b = np.sum(arr_b**2,axis = 1)

dists = a + b - 2*np.matmul(arr_a,arr_b.T)
dists[dists < 1e-6] = float(0.0)
dists = np.sqrt(dists)
type(np.min(dists,axis = np.argmin(dists.shape)))

numpy.ndarray

In [6]:
# Function based on all the computations above
def calcuate_distances(label_set_a:list, 
                       label_set_b:list) -> np.ndarray:
    """
    This function takes in two sets of glove embeddings vectors and returns the min distances between the two
    
    Parameters
    -------------    
    label_set_a : list 
            the first set of glove embedding vectors from one input source
    label_set_b : list
            the second set of glove embedding vectors from the second source
    
    Returns
    ---------
    numpy.ndarray
        The list of distances, where length = max(len(label_set_a),len(label_set_b))
    """
    
    #Turn both into numpy arrays
    arr_a = np.array(label_set_a)
    arr_b = np.array(label_set_b)
    
    #Square and transform as needed
    a = np.sum(arr_a**2,axis = 1)[:,np.newaxis]
    b = np.sum(arr_b**2,axis = 1)
    
    #Calculate the distances and take the square root
    #We are also cutting off where values too small
    dists = a + b - 2*np.matmul(arr_a,arr_b.T)
    dists[dists < 1e-6] = float(0.0)
    dists = np.sqrt(dists)
    
    #Return the minimum values across the axis with more glove embeddings
    return np.min(dists,axis = np.argmin(dists.shape))

In [7]:
# Function based on all the computations above
def calcuate_distance(label_set_a:list, 
                       label_set_b:list) -> np.ndarray:
    """
    This function takes in two sets of glove embeddings vectors and returns a single value representing the distance between the two values
    
    Parameters
    -------------    
    label_set_a : list 
            the first set of glove embedding vectors from one input source
    label_set_b : list
            the second set of glove embedding vectors from the second source
    
    Returns
    ---------
    float32
        A single value representing the distance between label_set_a and label_set_b
    """
    
    #Turn both into numpy arrays
    arr_a = np.array(label_set_a)
    arr_b = np.array(label_set_b)
    
    #Square and transform as needed
    a = np.sum(arr_a**2,axis = 1)[:,np.newaxis]
    b = np.sum(arr_b**2,axis = 1)
    
    #Calculate the distances and take the square root
    #We are also cutting off where values too small
    dists = a + b - 2*np.matmul(arr_a,arr_b.T)
    dists[dists < 1e-6] = float(0.0)
    dists = np.sqrt(dists)
    
    #Return the minimum values across the axis with more glove embeddings
    return np.sum(np.min(dists,axis = np.argmin(dists.shape)))

## Now let's use our above function with labels of 2 similar images and 1 different

For this test, we will simply **only use the sem.seg labels to see how well we can tell distances with that** (idea is that captions will use similar idea):
- [Similar Image A](https://cocodataset.org/#explore?id=5253) and [Similar Image B](https://cocodataset.org/#explore?id=277614) selected on coco site by clicking *stop sign* (stop) and *traffic light* (stoplight)
- [Different Image A](https://cocodataset.org/#explore?id=360877) selected on coco site by clicking *apple* and *chair*

Let's see the 3 way comparison test for distances:

In [8]:
#Code to manually look through available words
# asd = list(glove.vocab.keys())
# asd.sort()
#print(asd[360000:370000])
"traffic-sign" in glove

False

In [9]:
#List of representaion based on sem. seg labels
similar_image_a = [glove["stop"],glove["stoplight"]]
similar_image_b = [glove["stop"],glove["stoplight"],glove["train"],glove["clock"]]
different_image_a = [glove["person"],glove["bottle"],glove["banana"],glove["apple"],glove["chair"]]

In [10]:
print("Distances between similar_image_a, similar_image_b: ")
print(calcuate_distances(similar_image_a,similar_image_b))

print("\n")
print("Distances between similar_image_a, different_image_a: ")
print(calcuate_distances(similar_image_a,different_image_a))

print("\n")
print("Distances between similar_image_b, different_image_a: ")
print(calcuate_distances(similar_image_b,different_image_a))

print("\n")
print("Printing the sums of each:")
print(calcuate_distance(similar_image_a,similar_image_b))
print(calcuate_distance(similar_image_a,different_image_a))
print(calcuate_distance(similar_image_b,different_image_a))

Distances between similar_image_a, similar_image_b: 
[0.        0.        3.8940194 4.6098065]


Distances between similar_image_a, different_image_a: 
[4.951073  5.6056447 5.913838  6.0824575 5.8651605]


Distances between similar_image_b, different_image_a: 
[4.951073  5.6056447 5.913838  5.834715  4.8754997]


Printing the sums of each:
8.503826
28.418173
27.18077


#### Now we will try the same process as above but with scrambling some of the ordering (not making it nice and uniform across the diff. inputs)

In [11]:
#List of representaion based on sem. seg labels but with different ordering
similar_image_a = [glove["stoplight"], glove["stop"]]
similar_image_b = [glove["train"], glove["stop"], glove["stoplight"], glove["clock"]]
different_image_a = [glove["bottle"],glove["banana"],glove["person"],glove["chair"], glove["apple"]]

In [12]:
print("Distances between similar_image_a, similar_image_b: ")
print(calcuate_distances(similar_image_a,similar_image_b))

print("\n")
print("Distances between similar_image_a, different_image_a: ")
print(calcuate_distances(similar_image_a,different_image_a))

print("\n")
print("Distances between similar_image_b, different_image_a: ")
print(calcuate_distances(similar_image_b,different_image_a))

print("\n")
print("Printing the sums of each:")
print(calcuate_distance(similar_image_a,similar_image_b))
print(calcuate_distance(similar_image_a,different_image_a))
print(calcuate_distance(similar_image_b,different_image_a))

Distances between similar_image_a, similar_image_b: 
[3.8940194 0.        0.        4.6098065]


Distances between similar_image_a, different_image_a: 
[5.6056447 5.913838  4.951073  5.8651605 6.0824575]


Distances between similar_image_b, different_image_a: 
[5.6056447 5.913838  4.951073  4.8754997 5.834715 ]


Printing the sums of each:
8.503826
28.418175
27.18077


## Observations

As you can see from the cell above, the two *similar images* were close to each other, **and had similar distances to the different image** which can be really useful for **outlier analysis**. 

Another great thing that you can see is that *independent of the order of the different label input* **the outputs for the overall distances were identical** (and the vectors just in a different ordering, same values)

- We can also try this with higher dimensional (this was just with 50D vectors) so higher dimensional might lead to even tighter distances (closer things are closer, farther things are farther)
- We can also try this process and calculate distances **within 1 single image label set to find outlier labels** (test with scrambling labels)

# Labels within a Singular Object Distance

Now let's see what happens when we scramble labels within a single image and see if there is any possible way to identify that:

In [13]:
from scipy import spatial

#Labels from similar image b, with a refridgerator
test_labels = [glove["train"], glove["stop"], glove["stoplight"], glove["clock"],glove["refrigerator"]]

#Now let's see which one is the furthest from the others
empty_list =np.array([[0 for index in range(len(test_labels))] for index in range(len(test_labels))]).reshape(len(test_labels),-1)

for index in range(len(test_labels)):
    for sec_ind in range(len(test_labels)):
        print(test_labels[index])
        print(test_labels[sec_ind])
        print(1 - spatial.distance.cosine(test_labels[index], test_labels[sec_ind]))

print(empty_list)

[ 0.94971    0.34328    0.84504   -0.88519   -0.72078   -0.29309
 -0.74678    0.65122    0.47295   -0.74011    0.1877    -0.38279
 -0.55899    0.42952   -0.26984   -0.42383   -0.31236    1.3423
 -0.78567   -0.6302     0.91819    0.21126   -0.57442    1.4549
  0.75456   -1.6165    -0.0085015  0.0029134  0.51304   -0.47447
  2.5306     0.85944   -0.30667    0.057765   0.66231    0.20804
  0.64237   -0.5246    -0.053416   1.1404    -0.13703   -0.18361
  0.45459   -0.50963   -0.025539  -0.02861    0.18048   -0.4483
  0.40525   -0.36821  ]
[ 0.94971    0.34328    0.84504   -0.88519   -0.72078   -0.29309
 -0.74678    0.65122    0.47295   -0.74011    0.1877    -0.38279
 -0.55899    0.42952   -0.26984   -0.42383   -0.31236    1.3423
 -0.78567   -0.6302     0.91819    0.21126   -0.57442    1.4549
  0.75456   -1.6165    -0.0085015  0.0029134  0.51304   -0.47447
  2.5306     0.85944   -0.30667    0.057765   0.66231    0.20804
  0.64237   -0.5246    -0.053416   1.1404    -0.13703   -0.18361
  0.45

[ 0.94971    0.34328    0.84504   -0.88519   -0.72078   -0.29309
 -0.74678    0.65122    0.47295   -0.74011    0.1877    -0.38279
 -0.55899    0.42952   -0.26984   -0.42383   -0.31236    1.3423
 -0.78567   -0.6302     0.91819    0.21126   -0.57442    1.4549
  0.75456   -1.6165    -0.0085015  0.0029134  0.51304   -0.47447
  2.5306     0.85944   -0.30667    0.057765   0.66231    0.20804
  0.64237   -0.5246    -0.053416   1.1404    -0.13703   -0.18361
  0.45459   -0.50963   -0.025539  -0.02861    0.18048   -0.4483
  0.40525   -0.36821  ]
0.2701544761657715
[ 5.0625e-01 -7.5861e-02  3.5667e-01 -1.9916e-01  4.2828e-01 -4.9580e-01
  1.7988e-02  5.0164e-01 -7.4663e-02  1.6919e-02 -8.8081e-01 -1.6071e+00
  4.0707e-01  1.9289e-01 -3.0754e-01  1.1850e-01 -1.6106e-04  7.4773e-01
  1.3391e-01 -4.6638e-01 -6.7023e-01 -3.0118e-01 -3.0377e-01  1.2345e+00
  9.0100e-02  1.3800e-01 -3.3182e-01  1.4311e+00  9.8952e-01 -4.4896e-01
 -4.5327e-01  2.9784e-01 -5.6369e-01  8.7011e-01  1.0898e+00 -1.8810e-03
  

# Domain Generation

The cells below are specifically related to generating the terms related to the specific domain, which we can generate with a list of words:

```python
def gen_domain(list_words:list, depth:int) -> list:
```

In [18]:
def gen_domain(list_domain:list, depth:int) -> set:
    """
    This function takes in a list of strings, which represents the domain, and generates the relavent list of glove embeddings that represents this domain. 
    
    This does it through a recursive methodology
    
    
    Parameters
    -------------
    list_domain: list
    List of string terms that represent the domain
    
    depth: int
    How many layers should be used to generate the domain
    
    Returns
    ---------
    Set of strings of the terms that we should get the glove embeddings for
    """
    

    list_embeddings = set() #The final set representing the domain embeddings
    
    if depth == 0: #If we have gotten back to depth 0 it means we have added all the words to the depth we want
        return list_embeddings
    
    
    #We will use a BFS type function to generate our domain
    queue = set(list_domain)
    
    for word in queue: #For each domain word
        
        #Add the first thing in the queue to our list
        list_embeddings.add(word)
        
        #Get similar terms
        list_terms = set(word for word, similarity in glove.similar_by_vector(glove[word]))
        #Add its most similar words using a recursive call
        list_embeddings.update(gen_domain(list_terms, depth - 1))
    
    return list_embeddings    

Now let's make a more advanced version of the above, but where we try to make sure that the terms are similar to existing values in the domain.

The primary step in doing this is to define a **threshold** value that we only add values above that:
- Through basic visual analysis, both *0.7* and *0.75* can be tried for threshold values

In [75]:
def gen_thresh_domain(list_domain:list, depth:int, threshold=0.7) -> set:
    """
    This function takes in a list of strings, which represents the domain, and generates the relavent list of glove embeddings that represents this domain. 
    
    This does it through a recursive methodology and uses a threshold value to make sure that only relavent values are pulled up.
    
    
    Parameters
    -------------
    list_domain: list
    List of string terms that represent the domain
    
    depth: int
    How many layers should be used to generate the domain
    
    threshold: float
    The similarity threshold that we will only add values above, by default = 0.7
    
    Returns
    ---------
    Set of strings of the terms that we should get the glove embeddings for
    """
    

    list_embeddings = set() #The final set representing the domain embeddings
    
    if depth == 0: #If we have gotten back to depth 0 it means we have added all the words to the depth we want
        return list_embeddings
    
    
    #We will use a BFS type function to generate our domain
    queue = set(list_domain)
 
    for word in queue: #For each domain word
        
        #Add the first thing in the queue to our list
        list_embeddings.add(word)
        
        #Get similar terms
        list_terms = set(word for word, similarity in glove.similar_by_vector(glove[word]) if similarity >= threshold)
        
        #Add its most similar words using a recursive call
        list_embeddings.update(gen_thresh_domain(list_terms, depth - 1))
    
    return list_embeddings    

In [86]:
def convert_to_embeddings(domain:set) -> list:
    """
    Converts the domain of terms to a list of related embeddings
    
    Parameters
    -------------
    domain: set
    The set of terms that define the domain, each term is a string
    
    Returns
    ---------
    list of glove embeddings
    """
    [glove[word] for word in domain]

In [82]:
b = gen_thresh_domain(["parked","stoplight","car","road","train"], 4)

print(len(b))
#print(set.intersection(vals, vally))


441


Let's do a test with this **huge** domain and see how close the vectors we had above are (the three vectors in the section above)

We need to make sure that we **convert our set of terms to list of glove embeddings**

In [85]:
bb = [glove[word] for word in b]

print("Distances between similar_image_a, domain: ")
print(calcuate_distances(similar_image_a,bb))

print("\n")
print("Distances between different_image_a, domain: ")
print(calcuate_distances(different_image_a,bb))

print("\n")
print("Distances between similar_image_b, domain: ")
print(calcuate_distances(similar_image_b,bb))

print("\n")
print("Printing the sums of each:")
print(calcuate_distance(similar_image_a,bb))
print(calcuate_distance(different_image_a,bb))
print(calcuate_distance(similar_image_b,bb))

Distances between similar_image_a, domain: 
[5.7320595 5.899094  5.2985787 4.761699  5.49797   5.7219687 4.94679
 4.3577924 5.533149  5.2869487 4.7893186 4.635488  5.506524  5.2671046
 4.926396  4.602645  6.0605326 5.484503  5.917894  5.874986  5.365782
 4.687152  5.047559  5.17592   4.4671626 5.188517  6.014544  5.684256
 6.275339  6.350981  5.2931614 3.8142302 5.9793286 6.3264146 6.140288
 6.1451883 5.580888  5.3642445 4.9933434 3.0593383 5.3640456 5.7511044
 4.860906  5.28498   6.039373  4.3820386 5.0482616 4.055481  4.9976473
 5.1699286 4.8493547 4.9420285 5.422241  5.157782  6.191807  5.163472
 4.894477  3.7781808 5.1259575 4.762961  5.612661  5.4239016 4.091848
 5.011543  5.1868396 4.0997515 5.238389  6.026382  5.7968073 4.79307
 6.597129  4.1266146 5.7927084 3.871517  4.888527  6.2060256 5.9289746
 5.4320555 4.565611  5.742092  6.187445  5.3181906 5.2823615 5.5616446
 5.1992617 4.8310337 3.8317282 5.3142624 5.8615212 5.7289467 3.6959257
 3.9615397 5.3518834 4.5306916 5.2162857 5

In [87]:

print(calcuate_distance(similar_image_a,bb))
print(calcuate_distance(different_image_a,bb))
print(calcuate_distance(similar_image_b,bb))

2258.0144
2581.1177
2095.9043


In [88]:
abs(2258.0144 - 2581.1177)

323.10329999999976

In [89]:
abs(2258.0144 - 2095.9043)

162.11009999999987

In [90]:
abs(2581.1177 - 2095.9043)

485.21339999999964